In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
import json
import requests

from symbol import Symbol
from helpers import rename_columns, SafeHDFStore, clean_file
from collections import OrderedDict

import warnings
warnings.filterwarnings('ignore')

In [2]:
s = Symbol()
smeta = s.symbol_meta
codes = pd.read_hdf('edelweiss.h5', 'codes')
codes = smeta.join(codes)[['code']]
codes.count()

code    1476
dtype: int64

In [3]:

# edelweiss_link.to_hdf('edelweiss.h5', 'links')

Loaded Successfully for 20microns
Loaded Successfully for 3iinfotech
Loaded Successfully for 3mindia
Loaded Successfully for 63moons
Loaded Successfully for 8kmiles
Loaded Successfully for a2zinfra
Loaded Successfully for aartidrugs
Loaded Successfully for aartiind
Loaded Successfully for aarveeden


KeyboardInterrupt: 

In [17]:
edelweiss_link = pd.read_hdf('edelweiss.h5', 'links')
cp_data = pd.read_hdf('edelweiss.h5', 'basic_data')

In [5]:
def load_cmp_data(cp_dataframe, data):
    try:
        for exchange_data in data:
            if exchange_data['Exchange'] == 'NSE':
                mydata = exchange_data
    except Exception as e:
        print('Unable to load for {0} due to {1}'.format(symbol, e))
        return pd.Series()
    data_series = pd.Series(mydata)
    cp_dataframe.loc[symbol] = data_series
    return cp_dataframe

In [27]:
def load_balance_sheet(bs_data, bs, symbol):

    try:
        std_years = bs['Years']
    except:
        print('Years not recieved for {0}'.format(symbol))
        std_years = ['2016', '2015', '2014', '2013', '2012']

    try:
        cons_years = bs['YearsCnd']
    except:
        print('Years not recieved for {0}'.format(symbol))
        cons_years = ['2016', '2015', '2014', '2013', '2012']

    for bs_element in bs_elements:
        try:
            element_data = bs[bs_element]
        except:
            pass
        i ,j = 0, 0
        for yearly_data in element_data:
            if yearly_data['IsCnsld']:
                try:
                     bs_data.loc[(symbol, 'consolidated', bs_element), cons_years[i]] = list(yearly_data.values())[1]
                except:
                    pass
                i=i+1

            else:
                try:
                    bs_data.loc[(symbol, 'standalone', bs_element),  std_years[j]] = list(yearly_data.values())[1]
                except:
                    pass
                j=j+1
load_balance_sheet(bs_data, bs, symbol)

In [28]:
def load_key_ratios(kr_data, kr, symbol):

    try:
        std_years = kr['Years']
    except:
        print('Years not recieved for {0}'.format(symbol))
        std_years = ['2016', '2015', '2014', '2013', '2012']

    try:
        cons_years = kr['YearsCnd']
    except:
        print('Years not recieved for {0}'.format(symbol))
        cons_years = ['2016', '2015', '2014', '2013', '2012']

    for kr_element in kr_elements:
        element_data = kr[kr_element]
        i ,j = 0, 0
        for yearly_data in element_data:
            if yearly_data['IsConsolidated']:
                try:
                     kr_data.loc[(symbol, 'consolidated', kr_element), cons_years[i]] = list(yearly_data.values())[0]
                except:
                    pass
                i=i+1

            else:
                try:
                    kr_data.loc[(symbol, 'standalone', kr_element),  std_years[j]] = list(yearly_data.values())[0]
                except:
                    pass
                j=j+1

In [29]:
def load_cash_flow(cf_data, cf, symbol):

    try:
        std_years = cf['NotCnsdYrs']
    except:
        print('Years not recieved for {0}'.format(symbol))
        std_years = ['2016', '2015', '2014', '2013', '2012']

    try:
        cons_years = cf['isCnsdYrs']
    except:
        print('Years not recieved for {0}'.format(symbol))
        cons_years = ['2016', '2015', '2014', '2013', '2012']


    for cf_element in cf_elements:
        element_data = cf[cf_element]
        i ,j = 0, 0
        for yearly_data in element_data:
            if yearly_data['IsCnsld']:
                try:
                     cf_data.loc[(symbol, 'consolidated', cf_element), cons_years[i]] = list(yearly_data.values())[0]
                except:
                    pass
                i=i+1

            else:
                try:
                    cf_data.loc[(symbol, 'standalone', cf_element),  std_years[j]] = list(yearly_data.values())[0]
                except:
                    pass
                j=j+1

In [30]:
def load_profit_loss(pnly_data, pnl, symbol):
    try:
        std_years = pnl['Years_Y']
    except:
        print('Years not recieved for {0}'.format(symbol))
        std_years = ['2016', '2015', '2014', '2013', '2012']

    try:
        cons_years = pnl['Years_YCnd']
    except:
        print('Years not recieved for {0}'.format(symbol))
        cons_years = ['2016', '2015', '2014', '2013', '2012']

    for pnl_element in pnl_elements:
        element_data = pnl[pnl_element]
        i ,j = 0, 0
        for yearly_data in element_data:
            if yearly_data['TP'] != 'Yearly':
                continue
            if yearly_data['IsCnsld']:
                try:
                     pnly_data.loc[(symbol, 'consolidated', pnl_element), cons_years[i]] = list(yearly_data.values())[0]
                except:
                    pass
                i=i+1

            else:
                try:
                    pnly_data.loc[(symbol, 'standalone', pnl_element),  std_years[j]] = list(yearly_data.values())[0]
                except:
                    pass
                j=j+1

    # try:
    #     std_hy_years = pnl['Years_HY']
    #     std_hy_months = pnl['Months_HY']
    # except:
    #     print('Years not recieved for {0}'.format(symbol))
    #     std_hy_years = ['2016', '2016', '2015', '2015', '2014']
    #     std_hy_months =['Sep', 'Mar', 'Sep', 'Mar', 'Sep']

    # try:
    #     cons_years = pnl['Years_YCnd']
    # except:
    #     print('Years not recieved for {0}'.format(symbol))
    #     cons_years = ['2016', '2015', '2014', '2013', '2012']

    # for pnl_element in pnl_elements:
    #     element_data = pnl[pnl_element]
    #     i ,j = 0, 0
    #     for yearly_data in element_data:
    #         if yearly_data['TP'] != 'Yearly':
    #             continue
    #         if yearly_data['IsCnsld']:
    #             try:
    #                  pnly_data.loc[('infy', 'consolidated', pnl_element), cons_years[i]] = list(yearly_data.values())[0]
    #             except:
    #                 pass
    #             i=i+1

    #         else:
    #             try:
    #                 pnly_data.loc[('infy', 'standalone', pnl_element),  std_years[j]] = list(yearly_data.values())[0]
    #             except:
    #                 pass
    #             j=j+1



In [31]:
session = requests.session()
cp_dataframe = pd.DataFrame(index=smeta.index.copy(), columns=[
    'BookValue', 'ChangeInLTPAbsolute', 'ChangeInLTPPercentage',
    'ClosePreviousDay', 'CompanyCode', 'DtTime', 'E52WHigh', 'E52WHigh_3',
    'E52WHigh_5', 'E52WLow', 'E52WLow_3', 'E52WLow_5', 'Exchange',
    'HighDay', 'LTP', 'LowDay', 'M1Return', 'MA_100D', 'MA_200D', 'MA_30D',
    'MA_50D', 'MarketCap', 'MarketDivYield', 'MarketEPS', 'MarketPE',
    'OpenDay', 'R1', 'R2', 'RowID', 'S1', 'S2', 'Symbol', 'Timestamp',
    'Value', 'Volume', 'W1Return', 'Y1Return'
])

symbols = edelweiss_links.index
conso = ['consolidated', 'standalone']
years_list = [
    '2016', '2015', '2014', '2013', '2012',
    '2011', '2010', '2009', '2008', '2007',
    '2006', '2005', '2004', '2003', '2002',
    '2001', '2000'
]


bs_elements = ['Sc', 'Rs', 'Br', 'Td', 'Ol', 'Tl', 'Fa', 'Ce', 'Oa', 'CWIP', 'Inv', 'Ta']
bs_index = pd.MultiIndex.from_product([symbols, conso, bs_elements], names=['symbol', 'type','bs_element'])
bs_data = pd.DataFrame(index=bs_index, columns=years_list)

kr_elements = [
    'GPMP', 'CPMP', 'NPMP',
    'ROE', 'ROCE', 
    'DE', 'IC', 
    'ATR', 'ITR', 'DTR', 
    'EPS', 'BVPS', 'DPS', 'FV', 'OS', 
    'NII', 'CTI', 'PATNII', 'NPA', 'ROA', 'CTD', 'CAR', 'ADR', 'CASAR', 'PCR', 
    'PE', 'PB', 'EV'
]
kr_index = pd.MultiIndex.from_product([symbols, conso, kr_elements], names=['symbol', 'type','kr_element'])
kr_data = pd.DataFrame(index=kr_index, columns=years_list)

cf_elements = [
    'CFOA', 'CFIA', 'CFFA', 'NCE', 'CBOY', 'CEOY',
]
cf_index = pd.MultiIndex.from_product([symbols, conso, cf_elements], names=['symbol', 'type','cf_element'])
cf_data = pd.DataFrame(index=cf_index, columns=years_list)

pnl_elements = [
    'Sl', 'Ex', 'Ii', 'Nii', 'Nr', 'Pfll', 'Nrp', 'Npi',
    'Pat', 'De', 'Gp', 'Sga', 'Onop', 'Op', 'Opm', 'Oi',
    'Int', 'Dp', 'PBT', 'Tax', 'EPS', 'Div'
]
pnl_index = pd.MultiIndex.from_product([symbols, conso, pnl_elements], names=['symbol', 'type','pnl_element'])
pnly_data = pd.DataFrame(index=pnl_index, columns=years_list)

In [32]:

for symbol, code in  zip(edelweiss_link.index, edelweiss_link.code):
# for symbol, code in  zip(['infy'], ['2806']):    
    
    data_url = 'https://ewmw.edelweiss.in/api/Market/Process/Equity/{0}'.format(code)
    data_headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.5',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Host': 'ewmw.edelweiss.in',
        'Origin': 'https://www.edelweiss.in',
        'Referer': 'https://www.edelweiss.in/quotes/equity/Infosys-Ltd-2806',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'
    }
    response = session.get(data_url, headers=data_headers, verify=False, )
    if response.status_code != 200:
        print('Unable to load for {0}'.format(symbol))
        continue
    data = json.loads(response.json())
    basic_data = data['JsonData']['companyMarketPrice']
    bs = data['JsonData']['equityDetails']['BS']
    kr = data['JsonData']['equityDetails']['KR']
    cf = data['JsonData']['equityDetails']['CF']
    pnl = data['JsonData']['equityDetails']['PNL']

    
    load_cmp_data(cp_dataframe, basic_data)
    load_balance_sheet(bs_data, bs, symbol)
    load_key_ratios(kr_data, kr, symbol)
    load_cash_flow(cf_data, cf, symbol)
    load_profit_loss(pnly_data, pnl, symbol)
    print('Loaded price data successfully for {0}'.format(symbol))
# cp_dataframe = cp_dataframe.convert_objects(convert_numeric=True)
# cp_dataframe[~(cp_dataframe.Symbol.str.strip().str.lower() == cp_dataframe.index)] = np.nan
# cp_dataframe.to_hdf('edelweiss.h5', 'basic_data')

Loaded price data successfully for 20microns
Loaded price data successfully for 3iinfotech
Loaded price data successfully for 3mindia
Loaded price data successfully for 63moons
Loaded price data successfully for 8kmiles
Loaded price data successfully for a2zinfra
Loaded price data successfully for aartidrugs
Loaded price data successfully for aartiind
Loaded price data successfully for aarveeden
Loaded price data successfully for aban
Loaded price data successfully for abb
Loaded price data successfully for abbotindia
Loaded price data successfully for abfrl
Loaded price data successfully for abirlanuvo
Loaded price data successfully for acc
Loaded price data successfully for accelya
Loaded price data successfully for ace
Loaded price data successfully for adanient
Loaded price data successfully for adaniports
Loaded price data successfully for adanipower
Loaded price data successfully for adanitrans
Loaded price data successfully for adffoods
Loaded price data successfully for adhunik

ProxyError: HTTPSConnectionPool(host='ewmw.edelweiss.in', port=443): Max retries exceeded with url: /api/Market/Process/Equity/106 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 502 Fiddler - Connection Failed',)))

In [34]:
cp_dataframe.to_hdf('edelweiss.h5', 'prices')
bs_data.to_hdf('edelweiss.h5', 'balance_sheet')
kr_data.to_hdf('edelweiss.h5', 'key_ratios')
cf_data.to_hdf('edelweiss.h5', 'cash_flow')
pnly_data.to_hdf('edelweiss.h5', 'pnl_yearly')

In [ ]:
data['JsonData']['equityDetails'].keys()
# print(data['JsonData']['equityDetails']['Sector'])
# print(data['JsonData']['equityDetails']['Industry'])
# print(data['JsonData']['equityDetails']['SubIndustry'])
# print(data['JsonData']['equityDetails']['PBV'])
# print(data['JsonData']['equityDetails']['PRSH'])
# print(data['JsonData']['equityDetails']['FIISH'])
# print(data['JsonData']['equityDetails']['DIISH'])
# print(data['JsonData']['equityDetails']['DivYieldPercentage'])
# print(data['JsonData']['equityDetails']['EVToEBITDA'])
# print(data['JsonData']['equityDetails']['ReturnOnCapitalEmployedPercentage'])
# print(data['JsonData']['equityDetails']['DebtToEquityRatio'])
# print(data['JsonData']['equityDetails']['Side'])
# print(data['JsonData']['equityDetails']['Beta'])
# print(data['JsonData']['equityDetails']['NFOCode'])
# 

In [ ]:
liss = jss['log']['entries']
for ls in liss:
    if ls['request']['url'] == 'https://ewmw.edelweiss.in/api/Market/Process/GetEquitiesSymbolDetails':
        mys = ls['request']
mys

In [ ]:
listss= []
for head in mys['headers']:
     listss.append('"{0}" : "{1}"'.format(head.get('name'), head.get('value')))
import pprint
pprint.pprint('{'+', '.join(listss)+'}')

In [ ]:
json.dumps('{'+', '.join(listss)+'}')

In [ ]:
psd = requests.get('https://ewstream.edelweiss.in/signalr/start?transport=webSockets&clientProtocol=1.5&connectionToken=AQAAANCMnd8BFdERjHoAwE%2FCl%2BsBAAAAkehhiqPHgE6YOI6ZrMjuFAAAAAACAAAAAAAQZgAAAAEAACAAAAChZ39t6cbr7goyVqlnKxuAuP6tw%2BrH7pX%2BTK8TleDxVQAAAAAOgAAAAAIAACAAAADLyZF%2BQo0KHhoMFw6LIAm3YXivnGROiUHDS6010iFo9zAAAABW7PzYR1BhaQPzbMpSWH535F%2BremEjXjWiMgFs6AsuM3cIMtxUEr5ZlSoDH4zu%2BHlAAAAA8FKuyICT5OJahRdPQz3FsH2rgZ38%2F3sMFaUc7eLkDJqrb220ptDUH%2FK3pwOYkUIC4EYsvNXkhAkicS8DLt8pnQ%3D%3D&connectionData=%5B%7B%22name%22%3A%22stockfeedhub%22%7D%5D&_=1485003929994')

In [ ]:
psd.text